# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
from intergrax.llm.messages import ChatMessage
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.planning.engine_planner import EnginePlanner

# -------------------------------------------------------
# Global test constants
# -------------------------------------------------------

USER_ID = "demo-user-planner"
SESSION_ID = "sess_planner_only_001"

# -------------------------------------------------------
# Build LLM adapter and RuntimeConfig
# IMPORTANT: Replace build_llm_adapter() with your actual builder.
# This must be the same adapter/config used in your runtime.
# -------------------------------------------------------

llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=True,
    enable_websearch=True,
    tools_mode="auto",
    enable_user_longterm_memory=True,    
)

In [4]:
import inspect
import json
import re
from typing import Any, Tuple

from intergrax.runtime.drop_in_knowledge_mode.planning.execution_plan_schema import ExecutionPlan


def _extract_json_object(text: str) -> str:
    if not text or not str(text).strip():
        raise ValueError("Empty LLM output.")

    s = str(text).strip()
    if s.startswith("{") and s.endswith("}"):
        return s

    m = re.search(r"\{.*\}", s, flags=re.DOTALL)
    if not m:
        raise ValueError("No JSON object found in LLM output.")
    return m.group(0).strip()


def _normalize_llm_output(out: Any) -> str:
    if out is None:
        return ""

    if isinstance(out, str):
        return out

    # ChatMessage-like
    if hasattr(out, "content") and isinstance(getattr(out, "content", None), str):
        return out.content

    # list of messages
    if isinstance(out, list) and out:
        last = out[-1]
        if hasattr(last, "content") and isinstance(getattr(last, "content", None), str):
            return last.content
        return "\n".join(str(x) for x in out)

    # dict-like
    if isinstance(out, dict):
        c = out.get("content")
        if isinstance(c, str):
            return c

    return str(out)


def _try_validate_plan(json_text: str) -> Tuple[ExecutionPlan | None, str]:
    """
    Returns (plan, debug_text). debug_text includes first validation error + context.
    """
    try:
        plan = ExecutionPlan.model_validate_json(json_text)
        return plan, ""
    except Exception as e:
        # try decode to show "nearby" keys if possible
        dbg = [f"{type(e).__name__}: {e}"]
        try:
            obj = json.loads(json_text)
            dbg.append("Top-level keys: " + ", ".join(sorted(obj.keys())))
            steps = obj.get("steps")
            if isinstance(steps, list) and steps:
                dbg.append(f"steps_count={len(steps)}; first_step.action={steps[0].get('action')}; last_step.action={steps[-1].get('action')}")
        except Exception:
            pass
        return None, "\n".join(dbg)


def build_stepplanner_prompt(user_message: str) -> str:
    return f"""
You are a planning component.

Your task is to generate a valid JSON object of type ExecutionPlan.

Rules:
- Output JSON only. No markdown. No commentary.
- The JSON must conform to the ExecutionPlan schema.
- Do not invent extra fields.
- If information is missing or ambiguous, use mode="clarify" and output a single ASK_CLARIFYING_QUESTION step.
- Otherwise use mode="execute" and end with FINALIZE_ANSWER.

User request:
{user_message}
""".strip()


async def run_stepplanner_real(message: str, *, run_id: str = "stepplanner-smoke-001") -> ExecutionPlan | None:
    prompt = build_stepplanner_prompt(user_message=message)  # <- Twoja funkcja promptu

    msgs = [
        ChatMessage(role="system", content="You are a StepPlanner. Return JSON only."),
        ChatMessage(role="user", content=prompt),
    ]

    out = llm_adapter.generate_messages(msgs, run_id=run_id)
    if inspect.iscoroutine(out):
        out = await out

    text = _normalize_llm_output(out)
    json_text = _extract_json_object(text)

    plan, dbg = _try_validate_plan(json_text)

    print("\n=== RAW OUTPUT (first 400 chars) ===")
    print(text[:400])

    if plan is None:
        print("\n=== VALIDATION FAILED ===")
        print(dbg)
        print("\n=== JSON (FULL) ===")
        print(json_text)
        return None

    print("\n=== VALIDATED PLAN ===")
    print(f"mode={plan.mode}, steps={len(plan.steps)}, intent={plan.intent}")
    print("actions:", [s.action for s in plan.steps])
    return plan



# 1) Generic (no websearch)
await run_stepplanner_real("Explain how to implement an async retry strategy in Python for API calls.", run_id="sp-1")

# 2) Freshness (must include websearch BEFORE first draft)
await run_stepplanner_real("What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates.", run_id="sp-2")

# 3) Project architecture (prefer LTM, no websearch)
await run_stepplanner_real("In my Intergrax runtime, should StepPlanner be separate from EnginePlanner? Give recommendation based on our architecture decisions.", run_id="sp-3")



=== RAW OUTPUT (first 400 chars) ===
{
  "steps": [
    {
      "mode": "clarify",
      "description": "What is the purpose of implementing a retry strategy?"
    },
    {
      "mode": "execute",
      "step": "import requests and implement the retry logic using a library like tenacity"
    },
    {
      "mode": "clarify",
      "description": "What are the specific API call patterns that need to be retried?"
    }
  ],
  "status"

=== VALIDATION FAILED ===
ValidationError: 18 validation errors for ExecutionPlan
plan_id
  Field required [type=missing, input_value={'steps': [{'mode': 'clar...'status': 'in_progress'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
intent
  Field required [type=missing, input_value={'steps': [{'mode': 'clar...'status': 'in_progress'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
mode
  Field required [type=missing, input_value={'steps': [{'mode': 'clar...'s